In [1]:
import pandas as pd
import numpy as np
import json
import os
from scipy.stats import norm, percentileofscore
import re
import ast

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

In [2]:
file_name = "mumbai_u15_dpdl"
with open(f'{file_name}.json', 'r') as f:
    data = json.load(f)
    dt = data[0]['scouting_data']

dataframe = pd.DataFrame(dt)
dataframe.to_excel(f'{file_name}.xlsx', index=False)

In [4]:


def top_players_by_position(player_data, relevant_attributes, position, n):
    df_player_data = pd.read_excel(player_data).drop(columns=['maximum_sense_score', 'minimum_sense_score',
                                                              'average_sense_score', 'minimum_pp_score','total_absolute_score',
                                                              'average_preferred_position_score'])
    df2= df_player_data.loc[df_player_data["total_game_time"]>= 90]

    df= df2.drop(list(df2)[1:7], axis=1)
    
    df_relevant_attributes = pd.read_csv(relevant_attributes)
    df_relevant_attributes['relevant_attributes'] = df_relevant_attributes['relevant_attributes'].apply(lambda lst: [s + '_per_90_percentile' for s in ast.literal_eval(lst)])

    # df_relevant_attributes['relevant_attributes'] = df_relevant_attributes['relevant_attributes'].apply(lambda lst: [s + '_percentile' for s in ast.literal_eval(lst)])

    
    for col in df.columns:
        df[f'{col}_z_score'] = (df[col] - df[col].mean()) / df[col].std(ddof=0)
        
            #print(df)
        
            # Calculate percentiles from Z-scores for each column
    for col in df.columns:
        if col.endswith('_z_score'):
            percentile_col = col.replace('_z_score', '_percentile')
            # percentile_col = col.replace('_z_score', '_per_90_percentile')
            df[percentile_col] = norm.cdf(df[col]) * 100
        
            # Drop the Z-score columns if not needed
    df.drop(columns=[col for col in df.columns if col.endswith('_z_score')], inplace=True)
            #print(df)
    
    #df
    cols_to_use= (df.columns.difference(df2.columns)).tolist()
    cols_to_use.append("user_id")
    #print(cols_to_use)
    df3= df2.merge(df[cols_to_use], on='user_id', how='left')
    # print(list(df3))
    
    df3.drop(columns= "user_id_percentile",inplace= True)
    # df3.drop(columns= "user_id_per_90_percentile",inplace= True)
    
    def calculate_sum(df_relevant_attributes, df3, position):
        # Filter df_relevant_attributes for the specified position
        position_row = df_relevant_attributes[df_relevant_attributes['preferred_position'] == position]
    
        if position_row.empty:
            print(f"No columns specified for position {position}")
            return None
    
        # Get the list of columns for the specified position
        columns = position_row['relevant_attributes'].iloc[0]
        # print(columns)
    
        # Filter df3 for rows where the specified position is present
        filtered_rows = df3[df3['preferred_position'].apply(lambda x: position in x)]
        # display(filtered_rows.head())
        filtered_rows.to_excel('dump.xlsx', index=False)
    
        if filtered_rows.empty:
            print(f"No rows with position {position} found in data")
            return None
    
        # Calculate sum of specified columns
        filtered_rows['sum'] = filtered_rows[columns].sum(axis=1)
    
        return filtered_rows
    
    # Example usage
    #position = 'CENTER FORWARD'
    result_df = calculate_sum(df_relevant_attributes, df3, position)
    
    if result_df is not None:
        # Store the result in a new DataFrame and remove rows where position is not present
        result_df = result_df[result_df['preferred_position'].apply(lambda x: position in x)]
    
    df_filtered = result_df.sort_values(by='sum', ascending=False)
    #df_filtered
    
    # Get the top n IDs
    top_players = df_filtered.head(n)

    def get_relevant_columns(df_relevant_attributes, df2, position):
        relevant_attributes = df_relevant_attributes[df_relevant_attributes['preferred_position'] == position]['relevant_attributes'].iloc[0]
        relevant_columns = [attr.rstrip("_percentile") for attr in relevant_attributes]
        column_order = df2.iloc[:, :7].columns.tolist() + relevant_columns
        return column_order
    relevant_columns = get_relevant_columns(df_relevant_attributes, df2, position)
    top_players= top_players[relevant_columns]

    # top_players.to_csv("final_report\Top_Players_"+"DPDL_Mum_U13_"+position+".csv")
    top_players.to_csv(f"final_report\\{file_name}_{position}.csv", index=False)
        
    return top_players

relevant_att_df = pd.read_csv('relevant_attributes_positionwise.csv')
positions = list(relevant_att_df['preferred_position'])

for pos in positions:
    #call the function in the form top_players_by_position(path to scouting data, path to relevant attributes file, position of player, number of top players you need)
    top_players_by_position(f"{file_name}.xlsx", r"relevant_attributes_positionwise.csv", pos, 10)

# #call the function in the form top_players_by_position(path to scouting data, path to relevant attributes file, position of player, number of top players you need)
# top_players_by_position(f"{file_name}.xlsx", "relevant_attributes_positionwise.csv", "LEFT SIDE BACK", 10)

No rows with position LEFT FORWARD found in data


AttributeError: 'NoneType' object has no attribute 'sort_values'